## 2014 Fire Polygon

In this notebook we will find those pixels that lie within the Fire Polygon from 2014.  There was a fire in 2014 and Jessica provided a file of  the resulting fire polygon.  Advisors believe that the high frequency of short grass is due to  the effects of this fire.  We want to  find all those Tiles that reside within the Fire Polygon to test this theory.

In [1]:
import gdal
import os
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def MakeGeoPanda(DF):
    poly=[]
    for index,row in DF.iterrows():
        p=Polygon([(row['ulx'],row['lry']),(row['lrx'],row['lry']),(row['lrx'],row['uly']),(row['ulx'],row['uly'])])
        poly.append(p)
    DF_GP=gpd.GeoDataFrame({'geometry':poly,"Fuel2014":list(DF["Label"].values),'FuelFileIndex':list(DF['FuelFileIndex'].values),"TileName":list(DF["TileName"].values)})
    return DF_GP

In [2]:
#Import Tiles Reference File
Tiles=pd.read_csv('WIFIRE/Processed_Data/LabelledTiles_Fuel_EqualArea/Reference_Fuel_EqualArea_18May2017.csv')
Tiles.head()

,Unnamed: 0,FuelFileIndex,Label,Lat,Long,TileName,lrx,lry,ulx,uly
0,0,0,91,33.147027,-117.205855,/home/mcmccart/WIFIRE/Processed_Data/LabelledT...,-117.205703,33.146875,-117.206007,33.147179
1,1,1,1,33.147027,-117.205551,/home/mcmccart/WIFIRE/Processed_Data/LabelledT...,-117.205400,33.146875,-117.205703,33.147179
2,2,2,1,33.147027,-117.205248,/home/mcmccart/WIFIRE/Processed_Data/LabelledT...,-117.205096,33.146875,-117.205400,33.147179
3,3,3,91,33.147027,-117.204944,/home/mcmccart/WIFIRE/Processed_Data/LabelledT...,-117.204793,33.146875,-117.205096,33.147179
4,4,4,91,33.147027,-117.204641,/home/mcmccart/WIFIRE/Processed_Data/LabelledT...,-117.204489,33.146875,-117.204793,33.147179


In [3]:
#Create GeoPanda for Tiles
Tiles_GP=MakeGeoPanda(Tiles)
Tiles_GP.head()

NameError: name 'MakeGeoPanda' is not defined

In [ ]:
#Import 2014 Fire Polygon shapefile
Fire=gpd.read_file('ca_cocos_20140515_2344_dd83.shp')
Fire.head()

In [ ]:
#Find all "Tiles" that intersect with the Fire Polygon
Intersect = gpd.sjoin(Fire, Tiles_GP, how="inner", op='intersects')
Intersect.head()

In [ ]:
Intersect.shape

In [ ]:
#Check Labels
Intersect['Fuel2014'].value_counts()

In [ ]:
#Import Fuel "decoder ring"
DecodeFuel=pd.read_csv('WIFIRE/Data_labels/fuel_labels.csv')
DecodeFuel.columns=["filename",'id','fuelLabel']
DecodeFuel.head()

A=pd.merge(Intersect,DecodeFuel,left_on='Fuel2014',right_on='id',how='left')
A["fuelLabel"].value_counts()

In [ ]:
Out=pd.merge(Intersect[['FuelFileIndex']],Tiles,on='FuelFileIndex',how='left')
Out[['Long','Lat','Label']].to_csv('Fuel2014_FirePolygon.xyz',sep=' ', index=False, header=False)
gdal_str="/opt/anaconda3/bin/gdal_translate Fuel2014_FirePolygon.xyz Fuel2014_FirePolygon.tif"
os.system(gdal_str)

Out[['Long','Lat','FuelFileIndex']].to_csv('Fuel2014Index_FirePolygon.xyz',sep=' ', index=False, header=False)

-----
Want to make  Fire Polygon for the new  labels (made via clustering)

In [2]:
#Import Tiles Reference File
Tiles=pd.read_csv('WIFIRE/Processed_Data/LabelledTiles_Fuel_EqualArea/Reference_Fuel_EqualArea_18May2017.csv',dtype=object, index_col=0)
Tiles["FuelFileIndex"]=Tiles["FuelFileIndex"].apply(lambda x: int(x))
Tiles["lrx"]=Tiles["lrx"].apply(lambda x: float(x))
Tiles["lry"]=Tiles["lry"].apply(lambda x: float(x))
Tiles["ulx"]=Tiles["ulx"].apply(lambda x: float(x))
Tiles["uly"]=Tiles["uly"].apply(lambda x: float(x))
Tiles.head()

,FuelFileIndex,Label,Lat,Long,TileName,lrx,lry,ulx,uly
0,0,91,33.1470269678005991,-117.205854862079065,/home/mcmccart/WIFIRE/Processed_Data/LabelledT...,-117.205703,33.146875,-117.206007,33.147179
1,1,1,33.1470269678005991,-117.205551406658813,/home/mcmccart/WIFIRE/Processed_Data/LabelledT...,-117.205400,33.146875,-117.205703,33.147179
2,2,1,33.1470269678005991,-117.205247951238547,/home/mcmccart/WIFIRE/Processed_Data/LabelledT...,-117.205096,33.146875,-117.205400,33.147179
3,3,91,33.1470269678005991,-117.204944495818296,/home/mcmccart/WIFIRE/Processed_Data/LabelledT...,-117.204793,33.146875,-117.205096,33.147179
4,4,91,33.1470269678005991,-117.20464104039803,/home/mcmccart/WIFIRE/Processed_Data/LabelledT...,-117.204489,33.146875,-117.204793,33.147179


In [3]:
#Import Tiles Reference File (need new labels)
TilesNL=pd.read_csv('WIFIRE/Processed_Data/LabelledTiles_Fuel_EqualArea/Reference_Fuel_EqualArea_18May2017_LabelsFromClustering.csv')
#change labels to values
Switch={'G':'1','S':'2','UF':'3','UG':'4','US':'5','UUF':'6','U':'7','W':'8','B':'10'}
TilesNL["NewLabel_V"]=TilesNL["NewLabel"].apply(lambda x: Switch[x])
#TilesNL["FuelFileIndex"]=TilesNL["FuelFileIndex"].apply(lambda x: str(x))
TilesNL.head()

,Image,FuelFileIndex,NewLabel,NewLabel_V
0,WIFIRE/Processed_Data/LabelledTiles_Fuel_Equal...,90,B,10
1,WIFIRE/Processed_Data/LabelledTiles_Fuel_Equal...,104,B,10
2,WIFIRE/Processed_Data/LabelledTiles_Fuel_Equal...,202,B,10
3,WIFIRE/Processed_Data/LabelledTiles_Fuel_Equal...,378,B,10
4,WIFIRE/Processed_Data/LabelledTiles_Fuel_Equal...,409,B,10


In [4]:
print (Tiles["FuelFileIndex"].dtype)
print (TilesNL["FuelFileIndex"].dtype)

int64
int64


In [5]:
Tiles=pd.merge(Tiles, TilesNL,on='FuelFileIndex',how='left')
Tiles.head()

,FuelFileIndex,Label,Lat,Long,TileName,lrx,lry,ulx,uly,Image,NewLabel,NewLabel_V
0,0,91,33.1470269678005991,-117.205854862079065,/home/mcmccart/WIFIRE/Processed_Data/LabelledT...,-117.205703,33.146875,-117.206007,33.147179,WIFIRE/Processed_Data/LabelledTiles_Fuel_Equal...,U,7
1,1,1,33.1470269678005991,-117.205551406658813,/home/mcmccart/WIFIRE/Processed_Data/LabelledT...,-117.205400,33.146875,-117.205703,33.147179,WIFIRE/Processed_Data/LabelledTiles_Fuel_Equal...,U,7
2,2,1,33.1470269678005991,-117.205247951238547,/home/mcmccart/WIFIRE/Processed_Data/LabelledT...,-117.205096,33.146875,-117.205400,33.147179,WIFIRE/Processed_Data/LabelledTiles_Fuel_Equal...,U,7
3,3,91,33.1470269678005991,-117.204944495818296,/home/mcmccart/WIFIRE/Processed_Data/LabelledT...,-117.204793,33.146875,-117.205096,33.147179,WIFIRE/Processed_Data/LabelledTiles_Fuel_Equal...,U,7
4,4,91,33.1470269678005991,-117.20464104039803,/home/mcmccart/WIFIRE/Processed_Data/LabelledT...,-117.204489,33.146875,-117.204793,33.147179,WIFIRE/Processed_Data/LabelledTiles_Fuel_Equal...,U,7


In [6]:
def MakeGeoPanda(DF):
    poly=[]
    for index,row in DF.iterrows():
        p=Polygon([(row['ulx'],row['lry']),(row['lrx'],row['lry']),(row['lrx'],row['uly']),(row['ulx'],row['uly'])])
        poly.append(p)
    DF_GP=gpd.GeoDataFrame({'geometry':poly,"Fuel2014":list(DF["NewLabel_V"].values),'FuelFileIndex':list(DF['FuelFileIndex'].values),"TileName":list(DF["TileName"].values)})
    return DF_GP

In [7]:
#Create GeoPanda for Tiles
Tiles_GP=MakeGeoPanda(Tiles)
Tiles_GP.head()

,Fuel2014,FuelFileIndex,TileName,geometry
0,7,0,/home/mcmccart/WIFIRE/Processed_Data/LabelledT...,"POLYGON ((-117.2060065897892 33.1468753676844,..."
1,7,1,/home/mcmccart/WIFIRE/Processed_Data/LabelledT...,"POLYGON ((-117.2057031343689 33.1468753676844,..."
2,7,2,/home/mcmccart/WIFIRE/Processed_Data/LabelledT...,"POLYGON ((-117.2053996789487 33.1468753676844,..."
3,7,3,/home/mcmccart/WIFIRE/Processed_Data/LabelledT...,"POLYGON ((-117.2050962235284 33.1468753676844,..."
4,7,4,/home/mcmccart/WIFIRE/Processed_Data/LabelledT...,"POLYGON ((-117.2047927681082 33.1468753676844,..."


In [8]:
#Import 2014 Fire Polygon shapefile
Fire=gpd.read_file('ca_cocos_20140515_2344_dd83.shp')
Fire.head()

,geometry
0,"POLYGON ((-117.1522177730791 33.1245863604779,..."


In [9]:
#Find all "Tiles" that intersect with the Fire Polygon
Intersect = gpd.sjoin(Fire, Tiles_GP, how="inner", op='intersects')
Intersect.head()

/opt/anaconda3/lib/python3.5/site-packages/numpy/lib/function_base.py:2652: RuntimeWarning: invalid value encountered in ? (vectorized)
  outputs = ufunc(*inputs)


,geometry,index_right,Fuel2014,FuelFileIndex,TileName
0,"POLYGON ((-117.1522177730791 33.1245863604779,...",76511,2,76511,/home/mcmccart/WIFIRE/Processed_Data/LabelledT...
0,"POLYGON ((-117.1522177730791 33.1245863604779,...",76164,2,76164,/home/mcmccart/WIFIRE/Processed_Data/LabelledT...
0,"POLYGON ((-117.1522177730791 33.1245863604779,...",75817,2,75817,/home/mcmccart/WIFIRE/Processed_Data/LabelledT...
0,"POLYGON ((-117.1522177730791 33.1245863604779,...",75470,2,75470,/home/mcmccart/WIFIRE/Processed_Data/LabelledT...
0,"POLYGON ((-117.1522177730791 33.1245863604779,...",76512,2,76512,/home/mcmccart/WIFIRE/Processed_Data/LabelledT...


In [11]:
Out.head()

,FuelFileIndex,Label,Lat,Long,TileName,lrx,lry,ulx,uly,Image,NewLabel,NewLabel_V
0,76511,1,33.0803229166736443,-117.153963985214816,/home/mcmccart/WIFIRE/Processed_Data/LabelledT...,-117.153812,33.080171,-117.154116,33.080475,WIFIRE/Processed_Data/LabelledTiles_Fuel_Equal...,S,2
1,76164,1,33.0806261169060392,-117.153963985214816,/home/mcmccart/WIFIRE/Processed_Data/LabelledT...,-117.153812,33.080475,-117.154116,33.080778,WIFIRE/Processed_Data/LabelledTiles_Fuel_Equal...,S,2
2,75817,1,33.0809293171384411,-117.153963985214816,/home/mcmccart/WIFIRE/Processed_Data/LabelledT...,-117.153812,33.080778,-117.154116,33.081081,WIFIRE/Processed_Data/LabelledTiles_Fuel_Equal...,S,2
3,75470,1,33.0812325173708359,-117.153963985214816,/home/mcmccart/WIFIRE/Processed_Data/LabelledT...,-117.153812,33.081081,-117.154116,33.081384,WIFIRE/Processed_Data/LabelledTiles_Fuel_Equal...,S,2
4,76512,1,33.0803229166736443,-117.153660529794564,/home/mcmccart/WIFIRE/Processed_Data/LabelledT...,-117.153509,33.080171,-117.153812,33.080475,WIFIRE/Processed_Data/LabelledTiles_Fuel_Equal...,S,2


In [12]:
Out=pd.merge(Intersect[['FuelFileIndex']],Tiles,on='FuelFileIndex',how='left')
Out[['Long','Lat','NewLabel_V']].to_csv('Fuel2014_FirePolygon.xyz',sep=' ', index=False, header=False)
gdal_str="/opt/anaconda3/bin/gdal_translate Fuel2014_FirePolygon.xyz Fuel2014_FirePolygon.tif"
os.system(gdal_str)


256

In [2]:
#Import Tiles Reference File (need new labels)
TilesNL=pd.read_csv('WIFIRE/Processed_Data/LabelledTiles_Fuel_EqualArea/Reference_Fuel_EqualArea_18May2017_LabelsFromClustering.csv')
#change labels to values
Switch={'G':'1','S':'2','UF':'3','UG':'4','US':'5','UUF':'6','U':'7','W':'8','B':'10'}
TilesNL["NewLabel_V"]=TilesNL["NewLabel"].apply(lambda x: Switch[x])
TilesNL.head()

,Image,FuelFileIndex,NewLabel,NewLabel_V
0,WIFIRE/Processed_Data/LabelledTiles_Fuel_Equal...,90,B,10
1,WIFIRE/Processed_Data/LabelledTiles_Fuel_Equal...,104,B,10
2,WIFIRE/Processed_Data/LabelledTiles_Fuel_Equal...,202,B,10
3,WIFIRE/Processed_Data/LabelledTiles_Fuel_Equal...,378,B,10
4,WIFIRE/Processed_Data/LabelledTiles_Fuel_Equal...,409,B,10


In [14]:
TilesNL["FuelFileIndex"]=TilesNL["FuelFileIndex"].apply(lambda x: str(x))

In [15]:
def ImportPreserve(FileName,labelName):
    DF=pd.read_table(FileName, header=None)
    DF.columns=['Row']
    DF["Long"]=DF["Row"].apply(lambda x: x.split(" ")[0])
    DF["Lat"]=DF["Row"].apply(lambda x: x.split(" ")[1])
    DF[labelName]=DF["Row"].apply(lambda x: x.split(" ")[2])
    del DF["Row"]
    return DF

Ref=ImportPreserve('Fuel2014Index_FirePolygon.xyz','FuelFileIndex')

In [16]:
Out=pd.merge(Ref,TilesNL,on='FuelFileIndex',how='left')
Out[['Long','Lat','NewLabel_V']].to_csv('Fuel2014_FirePolygon_NewLabels.xyz',sep=' ', index=False, header=False)
Out.head()

,Long,Lat,FuelFileIndex,Image,NewLabel,NewLabel_V
0,-117.15426744063507,33.080322916673644,76510,WIFIRE/Processed_Data/LabelledTiles_Fuel_Equal...,S,2
1,-117.15426744063507,33.08062611690604,76163,WIFIRE/Processed_Data/LabelledTiles_Fuel_Equal...,S,2
2,-117.15426744063507,33.08092931713844,75816,WIFIRE/Processed_Data/LabelledTiles_Fuel_Equal...,S,2
3,-117.1539639852148,33.080322916673644,76511,WIFIRE/Processed_Data/LabelledTiles_Fuel_Equal...,S,2
4,-117.1539639852148,33.08062611690604,76164,WIFIRE/Processed_Data/LabelledTiles_Fuel_Equal...,S,2


In [17]:
gdal_str="/opt/anaconda3/bin/gdal_translate Fuel2014_FirePolygon_NewLabels.xyz Fuel2014_FirePolygon_NewLabels.tif"
os.system(gdal_str)

256

For some reason, only grabbing the intersecting Tiles will not grid and make a TIFF file

So, instead I will just make a TIFF for 2014 of our new fuel categories

In [2]:
#Import Tiles Reference File (need new labels)
TilesNL=pd.read_csv('WIFIRE/Processed_Data/LabelledTiles_Fuel_EqualArea/Reference_Fuel_EqualArea_18May2017_LabelsFromClustering.csv')
#change labels to values
Switch={'G':'1','S':'2','UF':'3','UG':'4','US':'5','UUF':'6','U':'7','W':'8','B':'10'}
TilesNL["NewLabel_V"]=TilesNL["NewLabel"].apply(lambda x: Switch[x])
TilesNL.head()

,Image,FuelFileIndex,NewLabel,NewLabel_V
0,WIFIRE/Processed_Data/LabelledTiles_Fuel_Equal...,90,B,10
1,WIFIRE/Processed_Data/LabelledTiles_Fuel_Equal...,104,B,10
2,WIFIRE/Processed_Data/LabelledTiles_Fuel_Equal...,202,B,10
3,WIFIRE/Processed_Data/LabelledTiles_Fuel_Equal...,378,B,10
4,WIFIRE/Processed_Data/LabelledTiles_Fuel_Equal...,409,B,10


In [3]:
#Import Tiles Reference File
Tiles=pd.read_csv('WIFIRE/Processed_Data/LabelledTiles_Fuel_EqualArea/Reference_Fuel_EqualArea_18May2017.csv',dtype=object, index_col=0)
Tiles["FuelFileIndex"]=Tiles["FuelFileIndex"].apply(lambda x: int(x))
Tiles.head()

,FuelFileIndex,Label,Lat,Long,TileName,lrx,lry,ulx,uly
0,0,91,33.1470269678005991,-117.205854862079065,/home/mcmccart/WIFIRE/Processed_Data/LabelledT...,-117.20570313436893,33.1468753676844,-117.2060065897892,33.1471785679168
1,1,1,33.1470269678005991,-117.205551406658813,/home/mcmccart/WIFIRE/Processed_Data/LabelledT...,-117.20539967894868,33.1468753676844,-117.20570313436895,33.1471785679168
2,2,1,33.1470269678005991,-117.205247951238547,/home/mcmccart/WIFIRE/Processed_Data/LabelledT...,-117.20509622352841,33.1468753676844,-117.20539967894868,33.1471785679168
3,3,91,33.1470269678005991,-117.204944495818296,/home/mcmccart/WIFIRE/Processed_Data/LabelledT...,-117.20479276810816,33.1468753676844,-117.20509622352843,33.1471785679168
4,4,91,33.1470269678005991,-117.20464104039803,/home/mcmccart/WIFIRE/Processed_Data/LabelledT...,-117.2044893126879,33.1468753676844,-117.20479276810816,33.1471785679168


In [4]:
Tiles=pd.merge(Tiles, TilesNL,on='FuelFileIndex',how='left')
Tiles.head()

,FuelFileIndex,Label,Lat,Long,TileName,lrx,lry,ulx,uly,Image,NewLabel,NewLabel_V
0,0,91,33.1470269678005991,-117.205854862079065,/home/mcmccart/WIFIRE/Processed_Data/LabelledT...,-117.20570313436893,33.1468753676844,-117.2060065897892,33.1471785679168,WIFIRE/Processed_Data/LabelledTiles_Fuel_Equal...,U,7
1,1,1,33.1470269678005991,-117.205551406658813,/home/mcmccart/WIFIRE/Processed_Data/LabelledT...,-117.20539967894868,33.1468753676844,-117.20570313436895,33.1471785679168,WIFIRE/Processed_Data/LabelledTiles_Fuel_Equal...,U,7
2,2,1,33.1470269678005991,-117.205247951238547,/home/mcmccart/WIFIRE/Processed_Data/LabelledT...,-117.20509622352841,33.1468753676844,-117.20539967894868,33.1471785679168,WIFIRE/Processed_Data/LabelledTiles_Fuel_Equal...,U,7
3,3,91,33.1470269678005991,-117.204944495818296,/home/mcmccart/WIFIRE/Processed_Data/LabelledT...,-117.20479276810816,33.1468753676844,-117.20509622352843,33.1471785679168,WIFIRE/Processed_Data/LabelledTiles_Fuel_Equal...,U,7
4,4,91,33.1470269678005991,-117.20464104039803,/home/mcmccart/WIFIRE/Processed_Data/LabelledT...,-117.2044893126879,33.1468753676844,-117.20479276810816,33.1471785679168,WIFIRE/Processed_Data/LabelledTiles_Fuel_Equal...,U,7


In [7]:
Tiles[['Long','Lat','NewLabel_V']].to_csv('Fuel2014_NewLabels.xyz',sep=' ', index=False, header=False)
gdal_str="/opt/anaconda3/bin/gdal_translate Fuel2014_NewLabels.xyz Fuel2014_NewLabels.tif"
os.system(gdal_str)

0

In [8]:
Tiles[['Long','Lat','Label']].to_csv('Fuel2014_OriginalLabels.xyz',sep=' ', index=False, header=False)
gdal_str="/opt/anaconda3/bin/gdal_translate Fuel2014_OriginalLabels.xyz Fuel2014_OriginalLabels.tif"
os.system(gdal_str)

0